In [2]:
class Node:
    
    node_list = list()
    file = None
    
    
    def __init__(self, name=None, ip=None, ty=None):
        self.name = name
        self.ip = ip
        self.type = ty
    def start_tcp_server(self):
        pass
    def start_udp_server(self):
        pass
    
    def __download_file(self, name:str):
        pass
    def __create_get_message(self):
        pass
    def __create_discovery_message(self):
        pass
    def __create_response_message(self):
        pass
    def __send_discover(self, discovery):
        pass
    def __update_list(self, node_list):
        pass
    def __serialize_data(self, data):
        pass
    def __deserialize_data(self, data):
        pass
    def __handle_response(self):
        pass
    def __save_file(self):
        pass
    def __chose_best(self):
        pass
    def __reassemble_file(self):
        pass
    

In [3]:
class NetWolf:
    """
    This is main class of NetWolf project
    date: 6/8/2020
    author: Amirhosein Amir Firouzkouhi ( 9528007)
    """
    
    
    def start(self):
        pass
    def stop(self):
        pass
    def __start_user_command(self):
        pass
    def __str__(self):
        return "Net wolf < version 1>"


In [4]:
class File:
    
    def __init__(self, name=None, path=None):
        self.name = name
        self.path = path
    
    def save_data(self, data):
        pass
    def close_file(self):
        pass
    def __str__(self):
        pass

In [5]:
class Report:
    
    def __init__(self,name=None,path=None):
        self.name = name
        self.path = path
        
    def open_report(self):
        pass
    def close_report(self):
        pass
    def write_line(self):
        pass
    def erase_line(self):
        pass

In [6]:
class Server:
    
    def __init__(self, port_number = None, ts = None):
        self.port_number = port_number
        self.ts = ts
    def start(self):
        raise NotImplementedError("Subclass must implement this abstract method")
    def stop(self):
        raise NotImplementedError("Subclass must implement this abstract method")
    def __client_handler(self, socket = None):
        raise NotImplementedError("Subclass must implement this abstract method")
    def __get_thread_service(self, ts = None, func = None):
        pass
        

In [7]:
class TcpServer(Server):
    
    def start(self):
        pass
    def stop(self):
        pass
    def __client_handler(self, socket = None):
        pass

In [13]:
class UdpServer(Server):
    
    def start(self):
        pass
    def stop(self):
        pass
    def __client_handler(self, socket = None):
        pass    

In [24]:
from concurrent.futures import ThreadPoolExecutor
from time import sleep
import _thread



    
    
    

